In [119]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [56]:
# embed words into a vector space
# initialize them randomly
# words move around in a vector space. goal is to align semantically similar words
#  in the vector space.
# embeddings let you infer locally, on out-of-distribution input during inference.
#  => generalize
# how do you train an encoder?
words = open("names.txt", "r").read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [57]:
# map characters
chars = ['.'] + sorted(list(set(''.join(words))))
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for s, i in stoi.items()}

In [84]:
# build dataset

context_length = 3
X, Y = [], []

for w in words[:]:
#    print(w)
    context = [0] * context_length # padded context
    for ch in w + '.':
        ix = stoi[ch] # the integer index of current ch
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # "crop", and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [129]:
g = torch.Generator().manual_seed(2147483647)
# 27 possible characters. (our tokens are characters.)
# cram them into 2d.
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100)) # catted first layers --> 100 middle layers
b1 = torch.randn(100) # bias
# create the output / final layer.
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad=True

In [152]:
for i in range(10000):

    # minibatching.
    ix = torch.randint(0, X.shape[0], (32,))
    
    emb = C[X[ix]] # indexing into C to get the embeddings
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100). 32 is the size of the dset. 100 is the intermediate layer.
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])
   # print(loss.item())
    
    for p in parameters:
        p.grad = None
    loss.backward()
    lr = 10**-4
    for p in parameters:
        p.data += -lr * p.grad

#    lri.append(lrexp[i])
#    lossi.append(loss.item())
    
print(loss.item())

2.1210861206054688


In [153]:
emb = C[X]
h = torch.tanh(emb.view(-1, 6)@W1+b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss.item()

2.314883232116699